In [3]:
import os
import sys
import json
import xgboost as xgb
from sklearn.metrics import make_scorer, cohen_kappa_score, balanced_accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer
import numpy as np

In [7]:
# Add the directory two levels up to sys.path
grandparent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.insert(0, grandparent_dir)

# Now you can import the module from the grandparent directory
try:
    from metrics import *
    print("Module 'metrics' imported successfully.")
except ModuleNotFoundError as e:
    print(f"Error: {e}")
    print("Ensure that 'metrics.py' is located in the grandparent directory.")

Module 'metrics' imported successfully.


In [2]:
MODEL_NAME = 'xgb'

In [3]:
# Load features and labels
X = np.load('../../features.npy')
y = np.load('../../labels.npy')

X.shape, y.shape

((8724, 2048), (8724,))

In [4]:
# Count the number of samples in each class
print('Number of samples in each class:')
print(np.unique(y, return_counts=True))

# Define class names
class_names = ["Few", "Many", "None"]
print('Class names:')
print(class_names)

Number of samples in each class:
(array([0, 1, 2]), array([6232,  256, 2236]))
Class names:
['Few', 'Many', 'None']


In [5]:
# Divide the data into 2 classes only (Normal and Abnormal)
y_binary = np.zeros(y.shape)
y_binary[y != 2] = 0 # Few and Many
y_binary[y == 2] = 1 # None

# Check the number of samples in each class
print('Number of samples in each class:')
print(np.unique(y_binary, return_counts=True))

# Define the new class names
binary_class_names = ["Abormal", "Normal"]
print('Binary class names:')
print(binary_class_names)

Number of samples in each class:
(array([0., 1.]), array([6488, 2236]))
Binary class names:
['Abormal', 'Normal']


In [6]:
# Define the parameter grid for XGBoost
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'min_child_weight': [1, 3, 5]
}

custom_score = custom_score = make_scorer(custom_scorer, greater_is_better=True, 
                           response_method='predict_proba'
                           )

In [7]:
import cupy as cp

# Convert your data to GPU
X_gpu = cp.array(X)
y_binary_gpu = cp.array(y_binary)

X_gpu_np = cp.asnumpy(X_gpu)
y_binary_gpu_np = cp.asnumpy(y_binary_gpu)

In [8]:
# Initialize the KNN classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softprob',
                                device='cuda:0',
                                num_class=2,
                                random_state=42)

# Define the cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_clf, 
    param_distributions=param_grid,
    n_iter=100, 
    cv=cv, 
    verbose=3, 
    random_state=42, 
    # n_jobs=4,
    scoring=custom_score
)

In [9]:
# random_search.fit(X_gpu_np, y_binary_gpu_np)

In [10]:
# # Save best parameters
# with open('dd_normal_vs_abnormal_best_params.json', 'w') as f:
#     json.dump(random_search.best_params_, f, indent=4)

In [11]:
# Get the best parameters
# best_params = random_search.best_params_
# print("Best parameters:", best_params)

# Load the best parameters from a json file
with open('dd_normal_vs_abnormal_best_params.json', 'r') as f:
    best_params = json.load(f)

# Perform 10-fold cross-validation using the best model
best_clsfr_nor_abn = xgb.XGBClassifier(objective='multi:softprob', num_class=2, random_state=42, **best_params)

In [12]:
fold_results = []

for fold, (train_index, val_index) in enumerate(cv.split(X, y_binary), 1):
    print(f"Fold {fold}")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y_binary[train_index], y_binary[val_index]
    
    best_clsfr_nor_abn.fit(X_train, y_train)
    y_pred = best_clsfr_nor_abn.predict(X_val)
    y_pred = y_pred.argmax(axis=1)
    y_prob = best_clsfr_nor_abn.predict_proba(X_val)
    
    # Calculate metrics
    accuracy, class_metrics, auc, f1, cm, avg_sensitivity, avg_specificity = calculate_metrics(y_val, y_pred, y_prob, num_classes=2)

    # Log metrics for this fold
    metrics = {
        'fold': fold,
        'val_accuracy': accuracy,
        'val_auc': auc,
        'val_f1': f1,
        'avg_sensitivity': avg_sensitivity,
        'avg_specificity': avg_specificity,
        **{f'class_{binary_class_names[i]}_sensitivity': metrics["sensitivity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{binary_class_names[i]}_specificity': metrics["specificity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{binary_class_names[i]}_f1': 2 * metrics["precision"] * metrics["sensitivity"] / (metrics["precision"] + metrics["sensitivity"]) for i, metrics in enumerate(class_metrics)},
    }
    
    custom_log(metrics, model_name=f'dd_{MODEL_NAME}_nor_abn', log_dir='logs_dd/nor_vs_abn/')
    print("Metrics for this fold:")
    print(metrics)

    # Plot confusion matrix for this fold
    plot_confusion_matrix(cm, class_names=binary_class_names, epoch_num=0, model_name=f'dd_{MODEL_NAME}_nor_abn', fold_num=fold, save_dir='figures_dd/nor_vs_abn/')
    
    fold_results.append(metrics)

# Calculate and print average results across all folds
avg_results = {key: np.mean([fold[key] for fold in fold_results if key in fold]) 
               for key in fold_results[0].keys() if key != 'fold'}

print("Average results across all folds:")
for key, value in avg_results.items(): 
    print(f"{key}: {value}")

# Log average results
custom_log(avg_results, model_name=f'dd_{MODEL_NAME}_average_nor_abn', log_dir='logs_dd/nor_vs_abn/')

Fold 1
Metrics for this fold:
{'fold': 1, 'val_accuracy': 0.9765042979942693, 'val_auc': 0.9938297776996446, 'val_f1': 0.9530355097365406, 'avg_sensitivity': 0.9614723046642055, 'avg_specificity': 0.9614723046642055, 'class_Abormal_sensitivity': 0.9922958397534669, 'class_Normal_sensitivity': 0.930648769574944, 'class_Abormal_specificity': 0.930648769574944, 'class_Normal_specificity': 0.9922958397534669, 'class_Abormal_f1': 0.9604841427475387, 'class_Normal_f1': 0.9604841427475387}
Fold 2
Metrics for this fold:
{'fold': 2, 'val_accuracy': 0.9724928366762178, 'val_auc': 0.9946363877657246, 'val_f1': 0.9463087248322147, 'avg_sensitivity': 0.9639093701202677, 'avg_specificity': 0.9639093701202677, 'class_Abormal_sensitivity': 0.9815100154083205, 'class_Normal_sensitivity': 0.9463087248322147, 'class_Abormal_specificity': 0.9463087248322147, 'class_Normal_specificity': 0.9815100154083205, 'class_Abormal_f1': 0.9635879885421248, 'class_Normal_f1': 0.9635879885421248}
Fold 3
Metrics for thi

In [13]:
# Predict the class Abnormal for each sample using the best model from the first stage
y_pred = best_clsfr_nor_abn.predict(X)
abnormal_indices = np.where(y_pred == 0)[0]
print(f"Number of predicted abnormal samples: {len(abnormal_indices)}")

Number of predicted abnormal samples: 8724


In [14]:
# Filter the dataset to include only the predicted Abnormal sample
X_pred_abnormal = X[abnormal_indices]
y_pred_abnormal = y[abnormal_indices]

# Check the number of samples in each class
print('Number of samples in each class:')
len(y_pred_abnormal), len(X_pred_abnormal)

Number of samples in each class:


(8724, 8724)

In [15]:
print(np.unique(y_pred_abnormal, return_counts=True))
normal_in_abnormal = y_pred_abnormal == 2  # 2 is the label for Normal
len(y_pred_abnormal[normal_in_abnormal])

(array([0, 1, 2]), array([6232,  256, 2236]))


2236

In [16]:
X_abnormal = X_pred_abnormal[~normal_in_abnormal]
y_abnormal = y_pred_abnormal[~normal_in_abnormal]

# Check the number of samples in each class
print('Number of samples in each class:')
np.unique(y_abnormal, return_counts=True)

Number of samples in each class:


(array([0, 1]), array([6232,  256]))

In [17]:
# Initialize RandomizedSearchCV
random_search_abn = RandomizedSearchCV(
    estimator=xgb_clf, 
    param_distributions=param_grid,
    n_iter=100, 
    cv=cv, 
    verbose=3 , 
    random_state=42, 
    # n_jobs=-1,
    scoring=custom_score
)

In [71]:
random_search_abn.fit(X_abnormal, y_abnormal)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/samo/dev/liver/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [16:11:50] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[CV 1/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=1, n_estimators=500, subsample=0.8;, score=1.000 total time=   4.4s
[CV 2/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=1, n_estimators=500, subsample=0.8;, score=0.995 total time=   3.7s
[CV 3/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=1, n_estimators=500, subsample=0.8;, score=0.990 total time=   3.8s
[CV 4/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=1, n_estimators=500, subsample=0.8;, score=0.863 total time=   3.2s
[CV 5/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=1, n_estimators=500, subsample=0.8;, score=0.988 total time=   3.7s
[CV 1/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=9, min_child_weight=1, n_estimators=500, subsample=0.8;, score=1.000 total time=   4.7s
[CV 2/5] END colsample

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           device='cuda:0',
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           impo...
                                           n_estimators=None, n_jobs=None,
                                           num_class=2, num_parallel_tree=None, ...),
                   n_iter=100,
                   param_distributions={'colsample_bytree': [0.8, 1.0],
                                        'gamma': [0, 0.1, 0.2],
                                        'learning_rate': [0.01, 0.1, 0.3],
                                        'max_depth': [3, 5, 7, 9],
                                        'min_child_weight': [1, 3, 5],
                                        'n_estimators': [100, 200, 500],
                                        'subsample': [0.8, 1.0]},
                   random_state=42,
                   scoring=make_scorer(custom_scorer, response_method='predict_proba'),
                   verbose=3)

In [72]:
# Save best parameters few vs many
with open('dd_few_vs_many_best_params.json', 'w') as f:
    json.dump(random_search_abn.best_params_, f, indent=4)

In [18]:
# Load the best parameters from a json file
with open('dd_few_vs_many_best_params.json', 'r') as f:
    best_params_abn = json.load(f)

best_clsfr_few_many = xgb.XGBClassifier(objective='multi:softprob', num_class=2, random_state=42, **best_params_abn)


abn_class_names = ["Few", "Many"]

In [19]:
fold_results = []

for fold, (train_index, val_index) in enumerate(cv.split(X_abnormal, y_abnormal), 1):
    print(f"Fold {fold}")
    X_train, X_val = X_abnormal[train_index], X_abnormal[val_index]
    y_train, y_val = y_abnormal[train_index], y_abnormal[val_index]
    
    best_clsfr_few_many.fit(X_train, y_train)
    y_pred = best_clsfr_few_many.predict(X_val)
    y_pred = y_pred.argmax(axis=1)
    y_prob = best_clsfr_few_many.predict_proba(X_val)
    
    # Calculate metrics
    accuracy, class_metrics, auc, f1, cm, avg_sensitivity, avg_specificity = calculate_metrics(y_val, y_pred, y_prob, num_classes=2)

    # Log metrics for this fold
    metrics = {
        'fold': fold,
        'val_accuracy': accuracy,
        'val_auc': auc,
        'val_f1': f1,
        'avg_sensitivity': avg_sensitivity,
        'avg_specificity': avg_specificity,
        **{f'class_{abn_class_names[i]}_sensitivity': metrics["sensitivity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{abn_class_names[i]}_specificity': metrics["specificity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{abn_class_names[i]}_f1': 2 * metrics["precision"] * metrics["sensitivity"] / (metrics["precision"] + metrics["sensitivity"]) for i, metrics in enumerate(class_metrics)},
    }
    
    custom_log(metrics, model_name=f'dd_{MODEL_NAME}_few_many', log_dir='logs_dd/few_vs_many/')
    print("Metrics for this fold:")
    print(metrics)

    # Plot confusion matrix for this fold
    plot_confusion_matrix(cm, class_names=abn_class_names, epoch_num=0, model_name=f'dd_{MODEL_NAME}_few_many', fold_num=fold, save_dir='figures_dd/few_vs_many/')
    
    fold_results.append(metrics)

# Calculate and print average results across all folds
avg_results = {key: np.mean([fold[key] for fold in fold_results if key in fold]) 
               for key in fold_results[0].keys() if key != 'fold'}

print("Average results across all folds:")
for key, value in avg_results.items(): 
    print(f"{key}: {value}")

# Log average results
custom_log(avg_results, model_name=f'dd_{MODEL_NAME}_few_many_average', log_dir='logs_dd/few_vs_many/')

Fold 1


Metrics for this fold:
{'fold': 1, 'val_accuracy': 1.0, 'val_auc': 1.0, 'val_f1': 1.0, 'avg_sensitivity': 1.0, 'avg_specificity': 1.0, 'class_Few_sensitivity': 1.0, 'class_Many_sensitivity': 1.0, 'class_Few_specificity': 1.0, 'class_Many_specificity': 1.0, 'class_Few_f1': 1.0, 'class_Many_f1': 1.0}
Fold 2
Metrics for this fold:
{'fold': 2, 'val_accuracy': 0.9992295839753467, 'val_auc': 0.999984275987861, 'val_f1': 0.9902912621359223, 'avg_sensitivity': 0.9995990376904571, 'avg_specificity': 0.9995990376904571, 'class_Few_sensitivity': 0.9991980753809142, 'class_Many_sensitivity': 1.0, 'class_Few_specificity': 1.0, 'class_Many_specificity': 0.9991980753809142, 'class_Few_f1': 0.9995988768551946, 'class_Many_f1': 0.9995988768551946}
Fold 3
Metrics for this fold:
{'fold': 3, 'val_accuracy': 0.9992295839753467, 'val_auc': 1.0, 'val_f1': 0.9904761904761905, 'avg_sensitivity': 0.9995987158908507, 'avg_specificity': 0.9995987158908507, 'class_Few_sensitivity': 0.9991974317817014, 'class_Many_

In [20]:
def combine_predictions(normal_abnormal_pred, few_many_pred):
    # Initialize final_pred with the same shape as normal_abnormal_pred, filled with 2 (Normal)
    final_pred = np.full(normal_abnormal_pred.shape, 2)
    
    # Create a mask where normal_abnormal_pred is 0 (Abnormal)
    abnormal_mask = normal_abnormal_pred == 0
    
    # Update final_pred where the mask is True with values from few_many_pred
    final_pred[abnormal_mask] = few_many_pred
    
    # Return the final combined predictions
    return final_pred

def combine_probabilities(normal_abnormal_prob, few_many_prob, abnormal_mask):
    # Initialize with probabilities for Normal class
    combined_prob = np.column_stack((np.zeros_like(normal_abnormal_prob[:, 0]), 
                                     np.zeros_like(normal_abnormal_prob[:, 0]), 
                                     normal_abnormal_prob[:, 1]))
    
    # Update probabilities for Abnormal (Few and Many) classes
    if few_many_prob.size > 0:
        combined_prob[abnormal_mask, 0] = few_many_prob[:, 0] * normal_abnormal_prob[abnormal_mask, 0]
        combined_prob[abnormal_mask, 1] = few_many_prob[:, 1] * normal_abnormal_prob[abnormal_mask, 0]
        combined_prob[abnormal_mask, 2] = 0  # Probability of being Normal for predicted Abnormal samples
    
    # Normalize probabilities
    row_sums = combined_prob.sum(axis=1)
    combined_prob /= row_sums[:, np.newaxis]
    
    return combined_prob

In [21]:
overall_true = []
overall_pred = []
overall_prob = []

for fold, (train_index, val_index) in enumerate(cv.split(X, y), 1):
    print(f"Fold {fold}")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # Stage 1: Normal vs Abnormal
    y_train_binary = (y_train == 2).astype(int)
    best_clsfr_nor_abn.fit(X_train, y_train_binary)
    y_pred_binary = best_clsfr_nor_abn.predict(X_val)
    y_pred_binary = y_pred_binary.argmax(axis=1)
    y_prob_binary = best_clsfr_nor_abn.predict_proba(X_val)
    
    # Stage 2: Few vs Many (only for predicted Abnormal samples)
    abnormal_mask_train = y_train != 2
    X_train_abnormal = X_train[abnormal_mask_train]
    y_train_abnormal = y_train[abnormal_mask_train]
    
    best_clsfr_few_many.fit(X_train_abnormal, y_train_abnormal)
    
    abnormal_mask_val = y_pred_binary == 0
    X_val_abnormal = X_val[abnormal_mask_val]
    
    if len(X_val_abnormal) > 0:
        y_pred_few_many = best_clsfr_few_many.predict(X_val_abnormal)
        y_pred_few_many = y_pred_few_many.argmax(axis=1)
        y_prob_few_many = best_clsfr_few_many.predict_proba(X_val_abnormal)
    else:
        y_pred_few_many = np.array([])
        y_prob_few_many = np.array([])
    
    # Combine predictions and probabilities
    y_pred_combined = combine_predictions(y_pred_binary, y_pred_few_many)
    y_prob_combined = combine_probabilities(y_prob_binary, y_prob_few_many, abnormal_mask_val)
    
    overall_true.extend(y_val)
    overall_pred.extend(y_pred_combined)
    overall_prob.extend(y_prob_combined)
    
    # Calculate and log metrics for this fold
    accuracy, class_metrics, auc, f1, cm, avg_sensitivity, avg_specificity = calculate_metrics(y_val, y_pred_combined, y_prob_combined, num_classes=3)
     
    metrics = {
        'fold': fold,
        'val_accuracy': accuracy,
        'val_auc': auc,
        'val_f1': f1,
        'avg_sensitivity': avg_sensitivity,
        'avg_specificity': avg_specificity,
        **{f'class_{class_names[i]}_sensitivity': metrics["sensitivity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{class_names[i]}_specificity': metrics["specificity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{class_names[i]}_f1': 2 * metrics["precision"] * metrics["sensitivity"] / (metrics["precision"] + metrics["sensitivity"]) for i, metrics in enumerate(class_metrics)},
    }
    
    custom_log(metrics, model_name=f'dd_{MODEL_NAME}_combined', log_dir='logs_dd/combined/')
    print("Metrics for this fold:")
    print(metrics)
    
    # Plot confusion matrix for this fold
    plot_confusion_matrix(cm, class_names=class_names, epoch_num=0, model_name=f'dd_{MODEL_NAME}_combined', fold_num=fold, save_dir='figures_dd/combined/')

# Calculate overall metrics
overall_true = np.array(overall_true)
overall_pred = np.array(overall_pred)
overall_prob = np.array(overall_prob)

accuracy, class_metrics, auc, f1, cm, avg_sensitivity, avg_specificity = calculate_metrics(overall_true, overall_pred, overall_prob, num_classes=3)

overall_metrics = {
    'overall_accuracy': accuracy,
    'overall_auc': auc,
    'overall_f1': f1,
    'overall_avg_sensitivity': avg_sensitivity,
    'overall_avg_specificity': avg_specificity,
    **{f'overall_class_{class_names[i]}_sensitivity': metrics["sensitivity"] for i, metrics in enumerate(class_metrics)},
    **{f'overall_class_{class_names[i]}_specificity': metrics["specificity"] for i, metrics in enumerate(class_metrics)},
    **{f'overall_class_{class_names[i]}_f1': 2 * metrics["precision"] * metrics["sensitivity"] / (metrics["precision"] + metrics["sensitivity"]) for i, metrics in enumerate(class_metrics)},
}

print("Overall metrics:")
for key, value in overall_metrics.items():
    print(f"{key}: {value}")

custom_log(overall_metrics, model_name=f'dd_{MODEL_NAME}_combined_overall', log_dir='logs_dd/combined/')

# Plot overall confusion matrix
plot_confusion_matrix(cm, class_names=class_names, epoch_num=0, model_name=f'dd_{MODEL_NAME}_combined_overall', save_dir='figures_dd/combined/') 

Fold 1
Metrics for this fold:
{'fold': 1, 'val_accuracy': 0.9879656160458453, 'val_auc': 0.9867475677519018, 'val_f1': 0.9879612895691304, 'avg_sensitivity': 0.9891240842301913, 'avg_specificity': 0.9900691621132708, 'class_Few_sensitivity': 0.991980753809142, 'class_Many_sensitivity': 1.0, 'class_None_sensitivity': 0.9753914988814317, 'class_Few_specificity': 0.9779116465863453, 'class_Many_specificity': 1.0, 'class_None_specificity': 0.9922958397534669, 'class_Few_f1': 0.9848959589312646, 'class_Many_f1': 1.0, 'class_None_f1': 0.9837710569834908}
Fold 2
Metrics for this fold:
{'fold': 2, 'val_accuracy': 0.9862464183381089, 'val_auc': 0.9887170009884302, 'val_f1': 0.9862601941541317, 'avg_sensitivity': 0.9888005635598516, 'avg_specificity': 0.9897713172074099, 'class_Few_sensitivity': 0.9887730553327987, 'class_Many_sensitivity': 1.0, 'class_None_sensitivity': 0.9776286353467561, 'class_Few_specificity': 0.9799196787148594, 'class_Many_specificity': 0.999409681227863, 'class_None_spec